In [7]:
import numpy as np
import pygame
import random
import math

BLUE  = (0,0,255)
WHITE = (255,255,255)
BLACK = (0,0,0)
RED   = (255,0,0)

coefficients         = {}
coefficients['eta0'] = 1.0000
coefficients['wght'] = 0.0196
coefficients['eta1'] = 3.9804
coefficients['eta2'] = 0.2157
coefficients['eta3'] = 3.2353
coefficients['eta4'] = 0.5098
params               = {}
params['migration_exists'] = True
params['migration_coeff']  = 0.15
params['number_of_iter']   = 100
params['use_saved_data']   = False

class swarm(): # CUSTOMIZABLE OBJECT
    def __init__(self, func, lowerbounds, upperbounds, number_of_particles=30, display=True):
        
        self.lowerbounds  = lowerbounds
        self.upperbounds  = upperbounds
        self.nop          = number_of_particles
        self.display      = display
        self.func         = func
        self._vel_max     = 20.0
        self._vel_min     = -20.0
        self.member       = {}
        
        if self.display:
            self.screen       = pygame.display.set_mode((1000, 1000))
            pygame.display.set_caption("Particle Swarm Optimization")
            self.screen.fill(WHITE)
                    
        for ii in range(self.nop):
            self.member[str(ii)] = {'center':(0,0)}
            self.member[str(ii)]['best_value'] = 10**32
            
            self.member[str(ii)]['position']      = {str(jj): np.random.random()*\
                                                    (self.upperbounds[jj]-self.lowerbounds[jj])+\
                                                     self.lowerbounds[jj] for jj in range(len(self.lowerbounds))}
            self.member[str(ii)]['best_position'] = self.member[str(ii)]['position']
            self.member[str(ii)]['velocity']      = {str(jj): np.random.random()\
                                                     for jj in range(len(self.lowerbounds))}
            self.member[str(ii)]['center'] = (int(np.round(self.member[str(ii)]['position']['0'])),
                                              int(np.round(self.member[str(ii)]['position']['1'])))
        self._evaluate_fitness()
        self._distance()
            
    def _evaluate_fitness(self):
        '''evaluates the fitness function for the members of the swarm'''
        self.fitness_values = {'_'           :{},
                               'sorted'      :{},
                               'best'        :{},
                               'best_sorted' :{}}
        for ii in range(self.nop):
            self.member[str(ii)]['fitness_value'] = self.func([self.member[str(ii)]\
                                                    ['position'][str(jj)] for jj in\
                                                    range(len(self.lowerbounds))])
            
            if self.member[str(ii)]['fitness_value'] < self.member[str(ii)]['best_value']:
                self.member[str(ii)]['best_value'] = self.member[str(ii)]['fitness_value']
                self.member[str(ii)]['best_position'] = self.member[str(ii)]['position']
        
            self.fitness_values['_'][str(ii)]    = self.member[str(ii)]['fitness_value']
            self.fitness_values['best'][str(ii)] = self.member[str(ii)]['best_value']

        self._sorted_fitness_value    = sorted(self.fitness_values['_'].items(), key = lambda kv:(kv[1], kv[0]))
        self.fitness_values['sorted'] = {str(self._sorted_fitness_value[ii][0]):self._sorted_fitness_value[ii][1]\
                                         for ii in range(self.nop)}
        
        self._sorted_best_value       = sorted(self.fitness_values['best'].items(), key = lambda kv:(kv[1], kv[0]))
        self.fitness_values['best_sorted'] = {str(self._sorted_best_value[ii][0]):self._sorted_best_value[ii][1]\
                                              for ii in range(self.nop)}
        
        self.best_position = {list(self.fitness_values['best_sorted'].keys())[ii]:\
                              self.member[list(self.fitness_values['best_sorted'].keys())[ii]]['best_position']\
                              for ii in range(self.nop)}
        
    def update(self): 
        '''updates the position and velocity of the members of the swarm'''
        for ii in range(self.nop):
            self.member[str(ii)]['delta']    = {str(jj): np.random.randint(-2,4) * np.random.random()\
                                                         for jj in range(len(self.lowerbounds))}
            
            self.member[str(ii)]['position'] = {str(jj): self.member[str(ii)]['position'][str(jj)] + \
                                                         self.member[str(ii)]['delta'][str(jj)]\
                                                         for jj in range(len(self.lowerbounds))}
            
            self.member[str(ii)]['center'] = (int(np.round(self.member[str(ii)]['position']['0'])),
                                              int(np.round(self.member[str(ii)]['position']['1'])))
            if self.display:
                pygame.draw.circle(self.screen,BLUE,self.member[str(ii)]['center'],2)
        
        self._evaluate_fitness()
        self._distance()
        
        if self.display:
            pygame.display.flip()     
            self.screen.fill(WHITE)
            
    def _distance(self):
        '''calculates the distances between the members of the swarm'''
        for ii in range(self.nop):
            self.member[str(ii)]['distance'] = {str(jj): {str(kk): self.member[str(ii)]['position'][str(kk)]-\
                                                         self.member[str(jj)]['position'][str(kk)]\
                                                         for kk in range(len(self.lowerbounds))}\
                                                         for jj in range(self.nop)}
            
            self.member[str(ii)]['distance_sorted'] = {str(jj):sorted(np.abs(self.member[str(ii)]['distance']\
                                                      [str(kk)][str(jj)]) for kk in range(self.nop)) for jj in\
                                                       range(len(self.lowerbounds))}
            
            self.member[str(ii)]['abs_distance'] = {str(jj): np.sqrt(math.fsum(self.member[str(ii)]['distance']\
                                                             [str(jj)][str(kk)]**2 for kk in\
                                                             range(len(self.lowerbounds)))) for jj in\
                                                             range(self.nop)}
            
            self.member[str(ii)]['abs_distance_sorted'] = [sorted(self.member[str(ii)]['abs_distance'].items(),\
                                                          key = lambda kv:(kv[1], kv[0]))[jj][1] for jj in\
                                                          range(self.nop)]
            
            self.member[str(ii)]['closest_neighbours'] = [sorted(self.member[str(ii)]['abs_distance'].items(),\
                                                          key = lambda kv:(kv[1], kv[0]))[jj][0] for jj in\
                                                          range(self.nop)]
            
            self.member[str(ii)]['closest_neighbours_fitness_values'] = {self.member[str(ii)]['closest_neighbours'][jj]:\
                                                                         self.member[self.member[str(ii)]['closest_neighbours'][jj]]['fitness_value']\
                                                                         for jj in range(self.nop)}
            
            self.member[str(ii)]['closest_neigbours_best'] = sorted(list(self.member[str(ii)]\
                                                                 ['closest_neighbours_fitness_values'].items())[1:5],
                                                                 key=lambda kv:(kv[1],kv[0]))[0][0]
            
            '''distance from the closest particle which has the best fitness value'''
            self.member[str(ii)]['distance_from_closest_best'] = self.member[str(ii)]['distance']\
                                                                [self.member[str(ii)]['closest_neigbours_best']]
    
    def optimize(self):
        '''PSO algoritm'''
        for ii in range(self.nop):
            for jj in range(self.lowerbounds):
                self.member[str(ii)]['velocity'][str(jj)] =  coefficients['eta0'] * (coefficients['wght'] * \
                                                             self.member[str(ii)]['velocity'][str(jj)]) + \
                                                             coefficients['eta1'] * np.random.random() * \
                                                             self.member[str(ii)]['best_position'][str(jj)] - \
                                                             self.member[str(ii)]['position'][str(jj)] + \
                                                             coefficients['eta2'] * np.random.random() * \
                                                             list(self.best_position.values())[0]['jj'] - \
                                                             self.member[str(ii)]['position'][str(jj)] + \
                                                             coefficients['eta3'] * np.random.random() * \
                                                             list(self.member[str[ii]]['distance_sorted'].values())[jj] + \
                                                             coefficients['eta4'] * np.random.random() * \
                                                             self.member[str(ii)]['distance_from_closest_best']
        
                if self.member[str(ii)]['velocity'][str(jj)] > self._vel_max:
                    self.member[str(ii)]['velocity'][str(jj)] = self._vel_max
                
                                                      
                                                             
    


In [8]:
# coefficients         = {}
# coefficients['eta0'] = 1.0000
# coefficients['wght'] = 0.0196
# coefficients['eta1'] = 3.9804
# coefficients['eta2'] = 0.2157
# coefficients['eta3'] = 3.2353
# coefficients['eta4'] = 0.5098
# params               = {}
# params['migration_exists'] = True
# params['migration_coeff']  = 0.15
# params['number_of_iter']   = 100
# params['use_saved_data']   = False


# velocity(ii,jj) = coeff * (w * velocity(ii,jj) ...
#                 + eta1 * random('unif',0,1) * (personal_best_pos(ii,jj) - position(ii,jj))...
#                 + eta2 * random('unif',0,1) * (global_best_position(:,jj) - position(ii,jj)))...
#                 + eta3 * random('unif',0,1) * (position(closest_particles(ii,(jj-1)*4+2),jj) - position(ii,jj))...
#                 + eta4 * random('unif',0,1) * (position(closest_best_particles_indices(ii,jj) ,jj) - position(ii,jj))...
#                 + eta5 * random('unif',0,1) * (average_position(1,jj) - position(ii,jj))...
#                 + eta6 * random('unif',0,1) * average_velocity(1,jj);
            
#             if(velocity(ii,jj)>vel_max)
#                 velocity(ii,jj) = vel_max;
#             elseif(velocity(ii,jj)<vel_min)
#                 velocity(ii,jj) = vel_min;
#             end
            
#             position(ii,jj) = position(ii,jj) + velocity(ii,jj);
            
#             if(position(ii,jj)>upper_boundaries(1,jj))
#                 position(ii,jj) = upper_boundaries(1,jj);
#             elseif(position(ii,jj)<lower_boundaries(1,jj))
#                 position(ii,jj) = lower_boundaries(1,jj);
#             end

In [9]:
def egg(*args):
    x1    = args[0][0]
    x2    = args[0][1]
    term1 = -(x2+47) * math.sin(math.sqrt(math.fabs(x2+x1/2+47)))
    term2 = -x1 * math.sin(math.sqrt(math.fabs(x1-(x2+47))))
    out   = term1 + term2
    return out

def griewank(*args):
    d    = len(args[0])
    sum  = 0
    prod = 1
    for ii in range(d):
        xi   = xx[ii]
        sum  = sum + xi**2/4000
        prod = prod * math.cos(xi/math.sqrt(ii+1))
    out = sum - prod + 1
    return out

def bukin6(*args):
    x1 = args[0][0]
    x2 = args[0][1]    
    term1 = 100 * math.sqrt((abs(x2-0.01*x1**2)))
    term2 = 0.01 * abs(x1+10)    
    out = term1 + term2
    return out

def ackley(*args):
    d    = len(args[0])
    c    = math.pi * 2
    b    = 0.20
    a    = 20
    
    sum1 = 0
    sum2 = 0
    
    for ii in range(d):
        xi   = args[0][ii]
        sum1 = sum1 + xi**2
        sum2 = sum2 + math.cos(c*xi)
    
    term1 = -a * math.exp(-b*math.sqrt(sum1/d))
    term2 = -math.exp(sum2/d)
    
    y = term1 + term2 + a + math.exp(1)
    
    return y

def drop(*args):
    x1 = args[0][0]
    x2 = args[0][1]
    frac1 = 1+math.cos(12*math.sqrt(x1**2 + x2**2))
    frac2 = 0.5*(x1**2 + x2**2) + 2
    out = -frac1/frac2
    return out

In [10]:
fitness_function             = {}
fitness_function['egg']      = egg
fitness_function['griewank'] = griewank
fitness_function['bukin6']   = bukin6
fitness_function['ackley']   = ackley
fitness_function['drop']     = drop

In [11]:
particles = swarm(func=fitness_function['egg'],lowerbounds=[-512,-512],upperbounds=[512,512],
                  number_of_particles=50,display=False)

In [14]:
particles._distance.__doc__

'calculates the distances between the members of the swarm'

In [15]:
for ii in range(10):
    particles.update()

In [16]:
sorted(list(particles.member['39']['closest_neighbours_fitness_values'].items())[1:5],key=lambda kv:(kv[1],kv[0]))[0][0]

'10'

In [17]:
sorted(list(particles.member['39']['closest_neighbours_fitness_values'].values())[1:5])[0]

-316.31978114477505

In [18]:
particles.member['39']['closest_neigbours_best']

'10'

In [19]:
particles.member['39']['distance_from_closest_best']

{'0': 55.03860125582872, '1': 146.356346495597}

In [20]:
particles.member['39']['distance'][particles.member['39']['closest_neigbours_best']]

{'0': 55.03860125582872, '1': 146.356346495597}

In [21]:
{particles.member['39']['closest_neighbours'][ii]:particles.member[particles.member['39']['closest_neighbours'][ii]]['fitness_value'] for ii in range(particles.nop)}

{'39': 450.5333355743352,
 '3': 352.91637298285696,
 '30': 261.23957080700256,
 '48': 35.074030824692386,
 '10': -316.31978114477505,
 '24': -320.0534099884395,
 '40': 388.19388936101643,
 '32': -227.25300515152287,
 '49': -285.1998561212894,
 '26': -153.31198041336788,
 '0': -380.3831753701883,
 '41': -9.380052442061732,
 '25': -194.0946303291259,
 '2': -86.58485833194976,
 '14': -470.9228570184266,
 '34': 335.577332693058,
 '20': -345.79939542699617,
 '35': 118.2386296675476,
 '18': 5.1404116620919496,
 '8': 105.54281865898527,
 '19': -37.77566526129451,
 '16': 11.119400864157432,
 '6': 6.989002289900263,
 '36': -331.2889084640679,
 '37': -7.096520686397346,
 '4': -103.34007413802314,
 '45': -376.51474424525355,
 '9': 760.3692242393624,
 '31': -296.56776091484227,
 '46': 242.214134246843,
 '12': 200.20663484793425,
 '17': 104.9870927067928,
 '27': -56.9034973426775,
 '33': -234.78188596072619,
 '11': 89.1366931496216,
 '15': 362.26569355384817,
 '5': -301.96965757430604,
 '47': 383.4

In [22]:
particles.member['3']['closest_neighbours']

['3',
 '39',
 '10',
 '30',
 '48',
 '24',
 '32',
 '49',
 '40',
 '26',
 '41',
 '25',
 '2',
 '14',
 '34',
 '35',
 '0',
 '18',
 '8',
 '20',
 '19',
 '16',
 '6',
 '36',
 '37',
 '4',
 '45',
 '9',
 '31',
 '12',
 '46',
 '17',
 '27',
 '33',
 '15',
 '11',
 '5',
 '47',
 '22',
 '13',
 '43',
 '28',
 '38',
 '7',
 '29',
 '23',
 '21',
 '1',
 '44',
 '42']

In [23]:
particles.member['3']['distance_sorted']

{'0': [0.0,
  2.622771310498365,
  9.660219914005268,
  21.373337756659225,
  28.51893073064923,
  32.608947726515,
  57.66137256632709,
  61.97238931515784,
  63.90485150419897,
  77.93741710606628,
  113.69438486021278,
  114.94931109696125,
  121.3549954398402,
  204.6951888101066,
  210.76182653884644,
  219.8676901073069,
  232.76769674275616,
  272.2180497611789,
  288.6373608441599,
  297.82321997200603,
  301.18938556354044,
  323.01971837009427,
  333.65222197627077,
  388.4499324428519,
  388.65575660286504,
  393.8648393202008,
  406.23397866043047,
  430.87279522649334,
  433.68924711727584,
  449.1943331983127,
  458.20918207120366,
  470.5883468282581,
  511.18460230017615,
  544.6335955533689,
  547.4306869748325,
  589.9033909589735,
  593.1572774244612,
  617.4323706082487,
  619.3399666627371,
  684.1579499377779,
  739.3426591875223,
  783.2615826252579,
  785.4866125133202,
  786.6548195537224,
  795.5266584069576,
  795.8257952408323,
  801.2585471404877,
  812.835

In [25]:
particles.member['39']

{'center': (436, 85),
 'best_value': 440.9497116965934,
 'position': {'0': 435.59261823474833, '1': 84.85092608311076},
 'best_position': {'0': 438.4620441177773, '1': 85.71974780755967},
 'velocity': {'0': 0.5037605446498972, '1': 0.0034595696613588522},
 'fitness_value': 450.5333355743352,
 'distance': {'0': {'0': -20.27604453973919, '1': -334.70260033056786},
  '1': {'0': 844.3161921200494, '1': 400.30565062295585},
  '2': {'0': 329.879422327506, '1': 99.39439875848244},
  '3': {'0': 57.66137256632709, '1': 23.24100330588047},
  '4': {'0': 488.5341677928204, '1': 219.63970543712645},
  '5': {'0': 741.819322504105, '1': 133.5817457018368},
  '6': {'0': 358.85075812986753, '1': 322.8997649833554},
  '7': {'0': 853.4871678071595, '1': -120.42539308943618},
  '8': {'0': 380.68109093642136, '1': 227.03933320190993},
  '9': {'0': -63.69362287351311, '1': 570.3379507284506},
  '10': {'0': 55.03860125582872, '1': 146.356346495597},
  '11': {'0': 647.5647635253007, '1': -336.15212165274465},

In [ ]:
list(particles.best_position.values())[0]

In [ ]:
particles.fitness_values['best_sorted']

In [ ]:
list(particles.fitness_values['best_sorted'].values())

In [ ]:
{list(particles.fitness_values['best_sorted'].keys())[ii]:particles.member[list(particles.fitness_values['best_sorted'].keys())[ii]]['best_position'] for ii in range(particles.nop)}

In [ ]:
particles.member[list(particles.fitness_values['best_sorted'].keys())[0]]['best_position']

In [ ]:
particles.member[list(particles.fitness_values['best_sorted'].keys())[0]]['best_value']

In [ ]:
particles.member['3']['distance_sorted']['0'][1]

In [ ]:
particles.member['3']['distance_sorted']['1'][1]

In [ ]:
sorted([particles.member['3']['distance'][str(ii)]['0'] for ii in range(particles.nop)])

In [ ]:
sorted([particles.member['3']['distance'][str(ii)]['1'] for ii in range(particles.nop)])

In [ ]:
{str(jj):sorted(np.abs(particles.member['3']['distance'][str(ii)][str(jj)]) for ii in range(particles.nop)) for jj in range(len(particles.lowerbounds))}

In [ ]:
particles.fitness_values['_']

In [ ]:
particles.fitness_values['sorted']

In [ ]:
particles.fitness_values['best']

In [ ]:
particles.fitness_values['best_sorted']

In [ ]:
list(particles.best_position.values())

In [ ]:
particles.fitness_values['best_sorted'][list(particles.fitness_values['best_sorted'].keys())[0]]

In [ ]:
fitness_function['egg']([440.9767564295708,442.35882834525336])

In [ ]:
coefficients         = {}
coefficients['eta0'] = 1.0000
coefficients['wght'] = 0.0196
coefficients['eta1'] = 3.9804
coefficients['eta2'] = 0.2157
coefficients['eta3'] = 3.2353
coefficients['eta4'] = 0.5098
params               = {}
params['migration_exists'] = True
params['migration_coeff']  = 0.15
params['number_of_iter']   = 100
params['use_saved_data']   = False

In [ ]:
# Particle Swarm Optimization (PSO) Code%% Girdiler
minimization = 1             # 0: maximization, 1: minimization
coeff        = 1.0           # katsayı // 1.0
w            = 0.0196        # weight of the partical velocity  //w    = 0.0196;
eta1         = 3.9804        # eta1 = 3.9804 // partical best
eta2         = 0.2157        # eta2 = 0.2157   // global best
eta3         = 3.2353        # eta3 = 3.2353  // closest neighbour
eta4         = 0.5098        # eta4 = 0.5098 // closest best neighbour
eta5         = 0.00          # eta5 = 0.00 // average position of the swarm
eta6         = 0.00          # eta6 = 0.00 // average velocity of the swarm
#fitness_function = @egg;    # the function that will be optimized
number_of_particles = 150    # The number of the elements (particles/bees/ants/birds) inside the swarm
number_of_variables = 2      # variable sayısı / dimension sayısı
migration           = 1      # 0: no-migration, 1: migration
pmig                = 0.15   # migration probability if migration is allowed // 0.15
mig_katsayi         = 0.0110 # migration katsayısı // = pmig (0.15)
objective_number    = 1      # the number of the objectives that the fitness function gives out
iteration           = 200    # Number of iterations
saved_data          = 1      # 1: use saved x_encoded values, 0: start randomly

In [ ]:
lower_boundaries = zeros(1,number_of_variables);
upper_boundaries = zeros(1,number_of_variables);

lower_boundaries(1,:) = [-512 -512];
upper_boundaries(1,:) = [512 512];
% lower_boundaries(1,:) = 0 * ones(1,number_of_variables);
% upper_boundaries(1,:) = 100 * ones(1,number_of_variables);
vel_min = -20;
vel_max = 20;

position = zeros(number_of_particles,number_of_variables);
velocity = zeros(number_of_particles,number_of_variables);
[row col] = size(position);          %%% size of the domain row: number of particles, col: number of variables
globalbest_value(1,1:col) = 1000000;

distances = zeros(number_of_particles,number_of_particles);
distances(1:number_of_particles,1:number_of_particles*number_of_variables) = 0;
closest_distances(1:number_of_particles,1:4*number_of_variables) = 1000000000;     %%% Her bir particle'�n kendisi dahil en yak�n 4 tane kom�usu i�in mesafeler bulunuyor.
closest_particles(1:number_of_particles,1:4*number_of_variables) = 0;

%%%% Kaydedilmi� veri kullan�lmadan, rastgele bir ver iseti ile ba�lan�yor %%%%

if(saved_data == 0)
    %%
    %%% Her bir partical i�in rastgele pozisyon (x1, x2, ..., xn)  ve h�z
    %%% atamas� yap�l�yor.
    for i = 1:row
        for j = 1:col
            position(i,j) = (upper_boundaries(1,j) - lower_boundaries(1,j)) * random('unif',0,1) + lower_boundaries(1,j);
            velocity(i,j) = random('unif',0,1);
        end
        fitness_values(i,:) = fitness_function(position(i,:));
    end
    personal_best_pos = position;
    
    total_velocity   = zeros(1,col);
    total_position   = zeros(1,col);
    average_velocity = zeros(1,col);
    average_position = zeros(1,col);
    for j = 1:col
        for i = 1:row
            total_velocity(1,j) = total_velocity(1,j) + velocity(i,j);
            total_position(1,j) = total_position(1,j) + position(i,j);
        end
        average_velocity(1,j) = total_velocity(1,j) / row;
        average_position(1,j) = total_position(1,j) / row;
    end
    
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%
    %%% Her bir partical'�n di�er partical'la ile aras�ndaki mesafe her bir variable i�in ayr� ayr� bulunuyor %%%
    for j = 1:col
        for i = 1:row
            for k = 1:row
                column = (j-1)*row + k;
                distances(i,column) = abs(position(i,j) - position(k,j));
            end
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%
    %%% Her bir particle'�n kendisi dahil en yak�n 4 tane kom�usu i�in
    %%% mesafeler ve partical indisleri bulunuyor. %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    for j = 1:col
        for i=1:row
            column = (j-1)*row ;
            sorted_neighbours(i,column+1:column+row) = sort(distances(i,column+1:column+row));
        end
    end
    
    for j = 1:col
        for i=1:row
            column = (j-1)*row ;
            sutun = (j-1) * 4;
            closest_distances(i,sutun+1:sutun+4) = sorted_neighbours(i,column+1:column+4);
        end
    end
    for j = 1:col
        for i=1:row
            for k=1:4
                column = (j-1)*row;
                sutun = (j-1) * 4 + k;
                closest_particles(i,sutun) = find(closest_distances(i,sutun) == distances(i,column+1:column+row),1,'first');
            end
        end
    end
    
    %%
    %%% Her bir partical i�in, o zaman kadar elde edilmi� en iyi fitness
    %%% de�eri kaydediliyor.
    for ii=1:row
        partical_best(ii,:) = fitness_function(position(ii,:));
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%%%%% En yak�n 4 particle'�n fitness de�erleri yaz�l�yor %%%%%%%%%%%%%
    for j = 1:col
        for i=1:row
            for k=1:4
                sutun = (j-1) * 4 + k;
                closest_particles_fitness_values(i,sutun) = fitness_values(closest_particles(i,sutun),1);
            end
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%%%% En yak�n 4 partical'�n fitness de�erleri ve particle'lar�n indisleri her bir variable i�in
    %%%%% en alttan en yukar�ya do�ru s�ralan�yor
    
    %%% En yak�n particle'lar�n fitness de�erleri s�ralan�yor
    closest_particles_fitness_values_sorted(:,:) = closest_particles_fitness_values(:,:);
    for j = 1:col
        for i=1:row
            sutun = (j-1) * 4 ;
            sorted_fitness_values(i,sutun+2:sutun+4) = sort(closest_particles_fitness_values(i,sutun+2:sutun+4));
        end
    end
    for j = 1:col
        for i=1:row
            sutun = (j-1) * 4 ;
            closest_particles_fitness_values_sorted(i,sutun+2:sutun+4) = sorted_fitness_values(i,sutun+2:sutun+4);
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%% En yak�n particle'lar�n indisleri s�ralan�yor
    for j = 1:col
        for i=1:row
            sutun = (j-1) * 4 ;
            closest_best_particles_indices(i,j) = closest_particles(i,find(closest_particles_fitness_values_sorted(i,sutun+2)==closest_particles_fitness_values(i,sutun+1:sutun+4),1,'first')+sutun);
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%
    %%%%% Global en iyi fitness de�eri bulunuyor
    if(minimization == 1)
        for ii=1:row
            if (partical_best(ii,:) < globalbest_value)
                globalbest_value = partical_best(ii,:);
                global_best_index = ii;
                global_best_position = position(global_best_index,:);
            end
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    genn = 1;
    %%
    %%% kaydedilmi� veri kullan�l�yor
else
    load('PSO_saved_data.mat');
    [saved_data_birey_sayisi saved_data_col] = size(fitness_values);
    genn = genn_saved;
    if(row>saved_data_birey_sayisi)
        for i = saved_data_birey_sayisi+1:row
            for j = 1:col
                position(i,j) = (upper_boundaries(1,j) - lower_boundaries(1,j)) * random('unif',0,1) + lower_boundaries(1,j);
                velocity(i,j) = random('unif',0,1);
            end
            fitness_values(i,:) = fitness_function(position(i,:));
            personal_best_pos(i,:) = position(i,:);
        end
        %%
        %%% Her bir partical'�n di�er partical'la ile aras�ndaki mesafe her bir variable i�in ayr� ayr� bulunuyor %%%
        for j = 1:col
            for i = 1:row
                for k = 1:row
                    column = (j-1)*row + k;
                    distances(i,column) = abs(position(i,j) - position(k,j));
                end
            end
        end
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        
        %%
        %%% Her bir particle'�n kendisi dahil en yak�n 4 tane kom�usu i�in
        %%% mesafeler ve partical indisleri bulunuyor. %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        for j = 1:col
            for i=1:row
                column = (j-1)*row ;
                sorted_neighbours(i,column+1:column+row) = sort(distances(i,column+1:column+row));
            end
        end
        
        for j = 1:col
            for i=1:row
                column = (j-1)*row ;
                sutun = (j-1) * 4;
                closest_distances(i,sutun+1:sutun+4) = sorted_neighbours(i,column+1:column+4);
            end
        end
        for j = 1:col
            for i=1:row
                for k=1:4
                    column = (j-1)*row;
                    sutun = (j-1) * 4 + k;
                    closest_particles(i,sutun) = find(closest_distances(i,sutun) == distances(i,column+1:column+row),1,'first');
                end
            end
        end
        
        %%
        %%% Her bir partical i�in, o zaman kadar elde edilmi� en iyi fitness
        %%% de�eri kaydediliyor.
        for ii=saved_data_birey_sayisi+1:row
            partical_best(ii,:) = fitness_function(position(ii,:));
        end
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        
        %%%%%% En yak�n 4 particle'�n fitness de�erleri yaz�l�yor %%%%%%%%%%%%%
        for j = 1:col
            for i=1:row
                for k=1:4
                    sutun = (j-1) * 4 + k;
                    closest_particles_fitness_values(i,sutun) = fitness_values(closest_particles(i,sutun),1);
                end
            end
        end
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        
        %%%%% En yak�n 4 partical'�n fitness de�erleri ve particle'lar�n indisleri her bir variable i�in
        %%%%% en alttan en yukar�ya do�ru s�ralan�yor
        
        %%% En yak�n particle'lar�n fitness de�erleri s�ralan�yor
        closest_particles_fitness_values_sorted(:,:) = closest_particles_fitness_values(:,:);
        for j = 1:col
            for i=1:row
                sutun = (j-1) * 4 ;
                sorted_fitness_values(i,sutun+2:sutun+4) = sort(closest_particles_fitness_values(i,sutun+2:sutun+4));
            end
        end
        for j = 1:col
            for i=1:row
                sutun = (j-1) * 4 ;
                closest_particles_fitness_values_sorted(i,sutun+2:sutun+4) = sorted_fitness_values(i,sutun+2:sutun+4);
            end
        end
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        
        %%% En yak�n particle'lar�n indisleri s�ralan�yor
        for j = 1:col
            for i=1:row
                sutun = (j-1) * 4 ;
                closest_best_particles_indices(i,j) = closest_particles(i,find(closest_particles_fitness_values_sorted(i,sutun+2)==closest_particles_fitness_values(i,sutun+1:sutun+4),1,'first')+sutun);
            end
        end
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        
        %%
        %%%%% Global en iyi fitness de�eri bulunuyor
        if(minimization == 1)
            for ii=1:row
                if (partical_best(ii,:) < globalbest_value)
                    globalbest_value = partical_best(ii,:);
                    global_best_index = ii;
                    global_best_position = position(global_best_index,:);
                end
            end
        end
        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    end
end

scatterplot(position(:,1:2));

iter = 0;

%%
%%%%%%% iterasyonlar ba�l�yor
while(iter < iteration)
    remainder = rem(iter,10);
    %%
    %%% Pozisyon ve h�z de�erleri g�ncelleniyor.
    for ii=1:row
        
        for jj=1:col
            velocity(ii,jj) = coeff * (w * velocity(ii,jj) ...
                + eta1 * random('unif',0,1) * (personal_best_pos(ii,jj) - position(ii,jj))...
                + eta2 * random('unif',0,1) * (global_best_position(:,jj) - position(ii,jj)))...
                + eta3 * random('unif',0,1) * (position(closest_particles(ii,(jj-1)*4+2),jj) - position(ii,jj))...
                + eta4 * random('unif',0,1) * (position(closest_best_particles_indices(ii,jj) ,jj) - position(ii,jj))...
                + eta5 * random('unif',0,1) * (average_position(1,jj) - position(ii,jj))...
                + eta6 * random('unif',0,1) * average_velocity(1,jj);
            
            if(velocity(ii,jj)>vel_max)
                velocity(ii,jj) = vel_max;
            elseif(velocity(ii,jj)<vel_min)
                velocity(ii,jj) = vel_min;
            end
            
            position(ii,jj) = position(ii,jj) + velocity(ii,jj);
            
            if(position(ii,jj)>upper_boundaries(1,jj))
                position(ii,jj) = upper_boundaries(1,jj);
            elseif(position(ii,jj)<lower_boundaries(1,jj))
                position(ii,jj) = lower_boundaries(1,jj);
            end
        end
        
        fitness_values(ii,:) = fitness_function(position(ii,:));
        
        if(fitness_values(ii,:) < partical_best(ii,:))
            partical_best(ii,:) = fitness_values(ii,:);
            personal_best_pos(ii,:) = position(ii,:);
        end
        
        if (partical_best(ii,:) < globalbest_value)
            globalbest_value = partical_best(ii,:);
            global_best_index = ii;
            global_best_position = position(global_best_index,:);
        end
        
        total_velocity   = zeros(1,col);
        total_position   = zeros(1,col);
        average_velocity = zeros(1,col);
        average_position = zeros(1,col);
        for j = 1:col
            for i = 1:row
                total_velocity(1,j) = total_velocity(1,j) + velocity(i,j);
                total_position(1,j) = total_position(1,j) + position(i,j);
            end
            average_velocity(1,j) = total_velocity(1,j) / row;
            average_position(1,j) = total_position(1,j) / row;
        end
    end
    %%
    %%%% Migration eklemesi yap�l�yor e�er migration=1 ise %%%
    if(migration==1)
        migration_probability = random('unif',0,1);
        total_migrating_particle_number = round(pmig*number_of_particles+objective_number);
        migrating_particle_number1 = round((pmig*number_of_particles+objective_number)/4);
        migrating_particle_number2 = 2 * migrating_particle_number1;
        migrating_particle_number3 = 3 * migrating_particle_number1;
        if(migration_probability <= pmig)
            sorted = sort(fitness_values(:,objective_number),'descend');
            
            %%% Fitness de�eri en d���k olan particle'lar�n indeksleri
            %%% bulunuyor
            for wi=1:total_migrating_particle_number
                worst_indeks(wi,1) = find(fitness_values(:,objective_number)==sorted(wi,1),1,'first');
            end
            
            %%% Fitness de�eri en y�ksek olan particle'lar�n indeksleri
            %%% bulunuyor
            for bi=1:(0.20*number_of_particles)
                best_indeks(bi,1) = find(fitness_values(:,objective_number)==sorted(number_of_particles+1-bi,1),1,'first');
                best_particle_array(bi,:) = position(best_indeks(bi,1),:);
            end
            
            %%% Fitness de�eri en y�ksek particle'lar�n average
            %%% pozisyonlar� bulunuyor.
            for j=1:col
                best_particles_average_position(1,j) = averaging(best_particle_array(:,j));
            end
            
            %%% En k�t� fitness de�erine sahip olanlar�n yar�s�, random
            %%% particle'lar ile de�i�tiriliyor
            for i = 1:migrating_particle_number1
                for j = 1:col
                    position(worst_indeks(i),j) = (upper_boundaries(1,j) - lower_boundaries(1,j)) * random('unif',0,1) + lower_boundaries(1,j);
                    velocity(worst_indeks(i),j) = random('unif',0,1);
                end
            end
            
            %%% En k�t� fitness de�erine sahip olanlar�n kalan yar�s�, en
            %%% iyi particle'lar�n ortalama pozisyon ve h�z de�eri merkez
            %%% al�narak random pozisyon ve h�z de�erlerine sahip yeni
            %%% particle'lar ile de�i�tiriliyor.
            for i = migrating_particle_number1+1:migrating_particle_number2
                for j = 1:col
                    sign = 1;
                    rastgelelik = random('unif',0,1);
                    if(rastgelelik < 0.50)
                        sign = -1;
                    end
                    position(worst_indeks(i),j) = 0.10 * (upper_boundaries(1,j) - lower_boundaries(1,j)) * random('unif',0,1) + sign * best_particles_average_position(1,j);
                    velocity(worst_indeks(i),j) = random('unif',0,1) * average_velocity(1,j);
                    
                    if(position(worst_indeks(i),j)>upper_boundaries(1,j))
                        position(worst_indeks(i),j) = upper_boundaries(1,j);
                    elseif(position(worst_indeks(i),j)<lower_boundaries(1,j))
                        position(worst_indeks(i),j) = lower_boundaries(1,j);
                    end
                    
                end
            end
            for i = migrating_particle_number2+1:migrating_particle_number3
                for j = 1:col
                    sign = 1;
                    rastgelelik = random('unif',0,1);
                    if(rastgelelik < 0.50)
                        sign = -1;
                    end
                    position(worst_indeks(i),j) = 10 * (best_particle_array(1,j) - best_particle_array(end,j)) * random('unif',0,1) + sign * best_particles_average_position(1,j);
                    velocity(worst_indeks(i),j) = random('unif',0,1) * average_velocity(1,j);
                    
                    if(position(worst_indeks(i),j)>upper_boundaries(1,j))
                        position(worst_indeks(i),j) = upper_boundaries(1,j);
                    elseif(position(worst_indeks(i),j)<lower_boundaries(1,j))
                        position(worst_indeks(i),j) = lower_boundaries(1,j);
                    end
                    
                end
            end
            for i = migrating_particle_number3+1:total_migrating_particle_number
                for j = 1:col
                    sign = 1;
                    rastgelelik = random('unif',0,1);
                    if(rastgelelik < 0.50)
                        sign = -1;
                    end
                    position(worst_indeks(i),j) = 2.0 * (best_particles_average_position(1,j) - average_position(1,j)) * random('unif',0,1) + sign * global_best_position(1,j);
                    velocity(worst_indeks(i),j) = random('unif',0,1) * average_velocity(1,j);
                    
                    if(position(worst_indeks(i),j)>upper_boundaries(1,j))
                        position(worst_indeks(i),j) = upper_boundaries(1,j);
                    elseif(position(worst_indeks(i),j)<lower_boundaries(1,j))
                        position(worst_indeks(i),j) = lower_boundaries(1,j);
                    end
                    
                end
            end
            
        end
        clear worst_indeks;
    end
    %%
    %%% Her bir partical'�n di�er partical'la ile aras�ndaki mesafe her bir variable i�in ayr� ayr� bulunuyor %%%
    for j = 1:col
        for i = 1:row
            for k = 1:row
                column = (j-1)*row + k;
                distances(i,column) = abs(position(i,j) - position(k,j));
            end
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%
    %%% Her bir particle'�n kendisi dahil en yak�n 4 tane kom�usu i�in
    %%% mesafeler ve partical indisleri bulunuyor. %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    for j = 1:col
        for i=1:row
            column = (j-1)*row ;
            sorted_neighbours(i,column+1:column+row) = sort(distances(i,column+1:column+row));
        end
    end
    
    for j = 1:col
        for i=1:row
            column = (j-1)*row ;
            sutun = (j-1) * 4;
            closest_distances(i,sutun+1:sutun+4) = sorted_neighbours(i,column+1:column+4);
        end
    end
    for j = 1:col
        for i=1:row
            for k=1:4
                column = (j-1)*row;
                sutun = (j-1) * 4 + k;
                closest_particles(i,sutun) = find(closest_distances(i,sutun) == distances(i,column+1:column+row),1,'first');
            end
        end
    end
    
    %%
    %%% Her bir partical i�in, o zaman kadar elde edilmi� en iyi fitness
    %%% de�eri kaydediliyor.
    for ii=1:row
        partical_best(ii,:) = fitness_function(position(ii,:));
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%%%%% En yak�n 4 particle'�n fitness de�erleri yaz�l�yor %%%%%%%%%%%%%
    for j = 1:col
        for i=1:row
            for k=1:4
                sutun = (j-1) * 4 + k;
                closest_particles_fitness_values(i,sutun) = fitness_values(closest_particles(i,sutun),1);
            end
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%%%% En yak�n 4 partical'�n fitness de�erleri ve particle'lar�n indisleri her bir variable i�in
    %%%%% en alttan en yukar�ya do�ru s�ralan�yor
    
    %%% En yak�n particle'lar�n fitness de�erleri s�ralan�yor
    closest_particles_fitness_values_sorted(:,:) = closest_particles_fitness_values(:,:);
    for j = 1:col
        for i=1:row
            sutun = (j-1) * 4 ;
            sorted_fitness_values(i,sutun+2:sutun+4) = sort(closest_particles_fitness_values(i,sutun+2:sutun+4));
        end
    end
    for j = 1:col
        for i=1:row
            sutun = (j-1) * 4 ;
            closest_particles_fitness_values_sorted(i,sutun+2:sutun+4) = sorted_fitness_values(i,sutun+2:sutun+4);
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    
    %%% En yak�n particle'lar�n indisleri s�ralan�yor
    for j = 1:col
        for i=1:row
            sutun = (j-1) * 4 ;
            closest_best_particles_indices(i,j) = closest_particles(i,find(closest_particles_fitness_values_sorted(i,sutun+2)==closest_particles_fitness_values(i,sutun+1:sutun+4),1,'first')+sutun);
        end
    end
    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    %%
    
    if (remainder == 1)
        scatterplot(position(:,1:2));
    end
    
    best_value_array(genn,1) = globalbest_value;
    
    genn = genn + 1;
    iter = iter + 1;
    text1 = 'iter = %d\n';
    fprintf(text1,iter);
    
end
genn_saved = genn;
save('PSO_saved_data.mat','position','velocity','closest_distances','closest_particles','globalbest_value','global_best_position','partical_best','personal_best_pos','closest_best_particles_indices','average_velocity','average_position','fitness_values','genn_saved','best_value_array');
close all;
scatterplot(position(:,1:2));